<a href="https://colab.research.google.com/github/Gagds/Arice/blob/main/AN_FA_ED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy.stats import ks_2samp
from collections import OrderedDict
from operator import itemgetter

from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, KFold
from sklearn.metrics import plot_confusion_matrix, recall_score, precision_score, plot_precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

from imblearn.over_sampling import SMOTE

In [109]:
dadosE = pd.read_csv('E.csv')
dadosF = pd.read_csv('F.csv')

In [110]:
dadosE.head()

,Nome,Data de Nascimento,Escolaridade,Ano de conclusão do Ensino Médio,Nota relacionada ao ano de conclusão do Ensino Médio,Nota da Prova,Qual o seu objetivo/propósito no cursinho?,Área de interesse,"Para aulas online, qual modelo você prefere?",Curso Pretendido,Instituição de preferência,"Nota (0,65*renda + 0,35*nota da prova)",Classificação
0,adna vieira carvalho,4/6/2003,EM completo,NaN,1,Não fez a prova,"Aprender não só pra entrar na faculdade, mas t...",Biológicas,Vídeo Gravado + PDF,----------,--------,Não fez a prova,Não fez a prova
1,adriana cristina oliveira germano,5/3/2003,NaN,NaN,1,Não fez a prova,NaN,NaN,Todos,medicina veterinária,NaN,Não fez a prova,Não fez a prova
2,adriano roberto de almeida júnior,1/17/2003,EM completo,2020.0,1,Não fez a prova,NaN,NaN,Vídeo Gravado,NaN,NaN,Não fez a prova,Não fez a prova
3,adrielli cristina cardoso pedroso,8/1/1993,EM completo,2013.0,1,Não fez a prova,Entrar na faculdade e prestar concursos,Exatas,Todos,Direito,não,Não fez a prova,Não fez a prova
4,albert dos santos nascimento,10/14/1998,EM completo,2018.0,1,Não fez a prova,Passar na faculdade,Humanas,Todos,NaN,NaN,Não fez a prova,Não fez a prova


In [111]:
dadosF.head()

,Nome,Filiação¹,Filiação²,Responsável,Quantas pessoas moram na casa?,1ºNome,1ºParentesco,1ºIdade,1ºProfissão/Ocupação,2ºNome,2ºParentesco,2ºIdade,2ºProfissão/Ocupação,Você tem apoio dos seus familiares?
0,adna vieira carvalho,Amélia Vieira Carvalho,Adineis Carvalho,Amélia Vieira Carvalho,4.0,Amélia Vieira Carvalho,Mãe,46.0,Diarista,NaN,NaN,NaN,NaN,Sim
1,adriana cristina oliveira germano,Robson Luis Germano,Andrezza Aparecida Oliveira Germano,Andrezza,4.0,Robson Luis Germano,Pai,41.0,Padeiro,Andrezza Aparecida Oliveira Germano,Mãe,39.0,Babá,NaN
2,adriano roberto de almeida júnior,adriano roberto de almeida,aleksandra juliana de faria almeida,aleksandra juliana de faria almeida,5.0,aleksandra juliana f almeida,Mãe,38.0,operadora de maquina,NaN,NaN,NaN,NaN,NaN
3,adrielli cristina cardoso pedroso,Dirce Cardozo Pedroso,José Francisco Pedroso,Dirce Cardozo Pedroso,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sim
4,albert dos santos nascimento,Luciene dos Santos,Alberto Francisco Nascimento,Luciene dos Santos,4.0,João Alvez Pereira,padrasto,61.0,Pedreiro,NaN,NaN,NaN,NaN,Sim


In [112]:
DadosA = pd.merge(dadosE, dadosF, how = 'left', on = 'Nome')

In [113]:
DadosA.head()

,Nome,Data de Nascimento,Escolaridade,Ano de conclusão do Ensino Médio,Nota relacionada ao ano de conclusão do Ensino Médio,Nota da Prova,Qual o seu objetivo/propósito no cursinho?,Área de interesse,"Para aulas online, qual modelo você prefere?",Curso Pretendido,Instituição de preferência,"Nota (0,65*renda + 0,35*nota da prova)",Classificação,Filiação¹,Filiação²,Responsável,Quantas pessoas moram na casa?,1ºNome,1ºParentesco,1ºIdade,1ºProfissão/Ocupação,2ºNome,2ºParentesco,2ºIdade,2ºProfissão/Ocupação,Você tem apoio dos seus familiares?
0,adna vieira carvalho,4/6/2003,EM completo,NaN,1,Não fez a prova,"Aprender não só pra entrar na faculdade, mas t...",Biológicas,Vídeo Gravado + PDF,----------,--------,Não fez a prova,Não fez a prova,Amélia Vieira Carvalho,Adineis Carvalho,Amélia Vieira Carvalho,4.0,Amélia Vieira Carvalho,Mãe,46.0,Diarista,NaN,NaN,NaN,NaN,Sim
1,adriana cristina oliveira germano,5/3/2003,NaN,NaN,1,Não fez a prova,NaN,NaN,Todos,medicina veterinária,NaN,Não fez a prova,Não fez a prova,Robson Luis Germano,Andrezza Aparecida Oliveira Germano,Andrezza,4.0,Robson Luis Germano,Pai,41.0,Padeiro,Andrezza Aparecida Oliveira Germano,Mãe,39.0,Babá,NaN
2,adriano roberto de almeida júnior,1/17/2003,EM completo,2020.0,1,Não fez a prova,NaN,NaN,Vídeo Gravado,NaN,NaN,Não fez a prova,Não fez a prova,adriano roberto de almeida,aleksandra juliana de faria almeida,aleksandra juliana de faria almeida,5.0,aleksandra juliana f almeida,Mãe,38.0,operadora de maquina,NaN,NaN,NaN,NaN,NaN
3,adrielli cristina cardoso pedroso,8/1/1993,EM completo,2013.0,1,Não fez a prova,Entrar na faculdade e prestar concursos,Exatas,Todos,Direito,não,Não fez a prova,Não fez a prova,Dirce Cardozo Pedroso,José Francisco Pedroso,Dirce Cardozo Pedroso,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sim
4,albert dos santos nascimento,10/14/1998,EM completo,2018.0,1,Não fez a prova,Passar na faculdade,Humanas,Todos,NaN,NaN,Não fez a prova,Não fez a prova,Luciene dos Santos,Alberto Francisco Nascimento,Luciene dos Santos,4.0,João Alvez Pereira,padrasto,61.0,Pedreiro,NaN,NaN,NaN,NaN,Sim


In [114]:
DadosA.fillna("NF")

,Nome,Data de Nascimento,Escolaridade,Ano de conclusão do Ensino Médio,Nota relacionada ao ano de conclusão do Ensino Médio,Nota da Prova,Qual o seu objetivo/propósito no cursinho?,Área de interesse,"Para aulas online, qual modelo você prefere?",Curso Pretendido,Instituição de preferência,"Nota (0,65*renda + 0,35*nota da prova)",Classificação,Filiação¹,Filiação²,Responsável,Quantas pessoas moram na casa?,1ºNome,1ºParentesco,1ºIdade,1ºProfissão/Ocupação,2ºNome,2ºParentesco,2ºIdade,2ºProfissão/Ocupação,Você tem apoio dos seus familiares?
0,adna vieira carvalho,4/6/2003,EM completo,NF,1,Não fez a prova,"Aprender não só pra entrar na faculdade, mas t...",Biológicas,Vídeo Gravado + PDF,----------,--------,Não fez a prova,Não fez a prova,Amélia Vieira Carvalho,Adineis Carvalho,Amélia Vieira Carvalho,4.0,Amélia Vieira Carvalho,Mãe,46,Diarista,NF,NF,NF,NF,Sim
1,adriana cristina oliveira germano,5/3/2003,NF,NF,1,Não fez a prova,NF,NF,Todos,medicina veterinária,NF,Não fez a prova,Não fez a prova,Robson Luis Germano,Andrezza Aparecida Oliveira Germano,Andrezza,4.0,Robson Luis Germano,Pai,41,Padeiro,Andrezza Aparecida Oliveira Germano,Mãe,39,Babá,NF
2,adriano roberto de almeida júnior,1/17/2003,EM completo,2020,1,Não fez a prova,NF,NF,Vídeo Gravado,NF,NF,Não fez a prova,Não fez a prova,adriano roberto de almeida,aleksandra juliana de faria almeida,aleksandra juliana de faria almeida,5.0,aleksandra juliana f almeida,Mãe,38,operadora de maquina,NF,NF,NF,NF,NF
3,adrielli cristina cardoso pedroso,8/1/1993,EM completo,2013,1,Não fez a prova,Entrar na faculdade e prestar concursos,Exatas,Todos,Direito,não,Não fez a prova,Não fez a prova,Dirce Cardozo Pedroso,José Francisco Pedroso,Dirce Cardozo Pedroso,3.0,NF,NF,NF,NF,NF,NF,NF,NF,Sim
4,albert dos santos nascimento,10/14/1998,EM completo,2018,1,Não fez a prova,Passar na faculdade,Humanas,Todos,NF,NF,Não fez a prova,Não fez a prova,Luciene dos Santos,Alberto Francisco Nascimento,Luciene dos Santos,4.0,João Alvez Pereira,padrasto,61,Pedreiro,NF,NF,NF,NF,Sim
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,vitória dos santos ribeiro de assis,9/29/2004,3° ano EM,2021,1,Não fez a prova,Procurar mais conhecimento,Exatas,Vídeo Gravado,Segurança pública e privada (tecnólogo em segu...,Uninter,Não fez a prova,Não fez a prova,Andreia Bento dos Santos,Claudinei Ribeiro de Assis,Andreia Bento dos Santos,3.0,NF,NF,NF,NF,NF,NF,NF,NF,Sim
76,wanessa domingos da silva,12/27/2000,EM completo,2018,1,Não fez a prova,Meu objetivo é conseguir passar na prova e faz...,Biológicas,PDF,Psicologia,Unisal,Não fez a prova,Não fez a prova,Maria Auxiliadora Francisco Dos Santos,Wanderley Domingos Da Silva,Maria Auxiliadora Francisco Dos Santos,5.0,Wanderson Domingos da Silva,Irmão,NF,Servente de Pedreiro,NF,NF,NF,NF,Sim
77,werik augusto da silva furtado\n,5/10/2002,EM completo,2019,1,Não fez a prova,Estudar o que não estudou no EM para poder pas...,Humanas,Vídeo Gravado,Fisioterapia,Não sabe ainda,Não fez a prova,Não fez a prova,Shyrlei Adriene da Silva Furtado,Paulo Sergio Furtado,Shyrlei Adriene da Silva Furtado,3.0,Paulo Sergio Furtado,Pai,52,Ajudante de Serviços Gerais,NF,NF,NF,NF,Não
78,willians renato ferreira,8/2/1982,Supletivo,2004,1,Não fez a prova,Evoluir; voltar a estudar e melhorar as condiç...,Humanas,Vídeo Gravado + PDF,Psicologia,USP/ UNICAMP/ Unisal,Não fez a prova,Não fez a prova,Sebastiana Ferreira,NF,Willians Renato Ferreira,6.0,NF,NF,NF,NF,NF,NF,NF,NF,Sim


In [115]:
DadosA.dtypes

Nome                                                                   object
Data de Nascimento                                                     object
Escolaridade                                                           object
Ano de conclusão do Ensino Médio                                      float64
Nota relacionada ao ano de conclusão do Ensino Médio                    int64
Nota da Prova                                                          object
Qual o seu objetivo/propósito no cursinho?                             object
Área de interesse                                                      object
Para aulas online, qual modelo você prefere?                           object
Curso Pretendido                                                       object
Instituição de preferência                                             object
Nota (0,65*renda + 0,35*nota da prova)                                 object
Classificação                                                   

In [116]:
#DadosA['Nota da Prova']=DadosA['Nota da Prova'].apply(lambda x: float(str(x).replace(',','.')))

In [117]:
DadosA.columns.values

array(['Nome', 'Data de Nascimento ', 'Escolaridade',
       'Ano de conclusão do Ensino Médio',
       'Nota relacionada ao ano de conclusão do Ensino Médio ',
       'Nota da Prova', 'Qual o seu objetivo/propósito no cursinho?',
       'Área de interesse ',
       'Para aulas online, qual modelo você prefere?                      ',
       'Curso Pretendido', 'Instituição de preferência',
       'Nota (0,65*renda + 0,35*nota da prova)', 'Classificação',
       'Filiação¹', 'Filiação²', 'Responsável',
       'Quantas pessoas moram na casa?', '1ºNome', '1ºParentesco',
       '1ºIdade', '1ºProfissão/Ocupação', '2ºNome', '2ºParentesco',
       '2ºIdade', '2ºProfissão/Ocupação',
       'Você tem apoio dos seus familiares? '], dtype=object)

In [217]:
CA=['Nome','1ºIdade','2ºIdade','1ºProfissão/Ocupação','2ºProfissão/Ocupação','Escolaridade','Ano de conclusão do Ensino Médio','Quantas pessoas moram na casa?','Você tem apoio dos seus familiares? ','Área de interesse ','1ºParentesco','2ºParentesco']

In [218]:
DS = DadosA.filter(items = CA)

In [185]:
DS.columns.values

array(['Nome', '1ºIdade', '2ºIdade', 'Escolaridade',
       'Ano de conclusão do Ensino Médio',
       'Quantas pessoas moram na casa?',
       'Você tem apoio dos seus familiares? ', 'Área de interesse ',
       '1ºParentesco', '2ºParentesco'], dtype=object)

In [120]:
DS.head

<bound method NDFrame.head of                                      Nome  1ºIdade  ...  1ºParentesco 2ºParentesco
0                    adna vieira carvalho     46.0  ...           Mãe          NaN
1       adriana cristina oliveira germano     41.0  ...           Pai          Mãe
2       adriano roberto de almeida júnior     38.0  ...           Mãe          NaN
3       adrielli cristina cardoso pedroso      NaN  ...           NaN          NaN
4            albert dos santos nascimento     61.0  ...      padrasto          NaN
..                                    ...      ...  ...           ...          ...
75    vitória dos santos ribeiro de assis      NaN  ...           NaN          NaN
76              wanessa domingos da silva      NaN  ...         Irmão          NaN
77       werik augusto da silva furtado\n     52.0  ...           Pai          NaN
78              willians renato ferreira       NaN  ...           NaN          NaN
79  yasmim cristine de oliveira carvalho      39.0  ...  

In [178]:
NV=(DS.filter(items=['Nome','Quantas pessoas moram na casa?']).groupby('Quantas pessoas moram na casa?' ).count())

In [179]:
NV.head(20)

,Nome
Quantas pessoas moram na casa?,
2.0,1
3.0,21
4.0,31
5.0,20
6.0,6
7.0,1


In [165]:
NS=(DS.filter(items=['Nome','Ano de conclusão do Ensino Médio']).groupby('Ano de conclusão do Ensino Médio' ).count())

In [166]:
NS.head(20)

,Nome
Ano de conclusão do Ensino Médio,
1997.0,1
2004.0,1
2009.0,2
2011.0,1
2012.0,1
2013.0,1
2016.0,1
2017.0,3
2018.0,7


In [159]:
NK=(DS.filter(items=['Nome','Você tem apoio dos seus familiares? ','Quantas pessoas moram na casa?']).groupby(['Você tem apoio dos seus familiares? '] ).describe())

In [161]:
NK.head(20)

Quantas pessoas moram na casa?  ...     
                                                              count  ...  max
Você tem apoio dos seus familiares?                                  ...     
Não                                                             7.0  ...  6.0
Sim                                                            64.0  ...  7.0

[2 rows x 8 columns]

In [ ]:
#ND=(DS.filter(items=['Nome','1ºIdade','2ºIdade']).groupby('1ºIdade').count().where(DS.Você tem apoio dos seus familiares? == 'Sim'	  ))

In [180]:
NV=(DS.filter(items=['Escolaridade','Quantas pessoas moram na casa?']).groupby(['Escolaridade'] ).describe())

In [181]:

NV.head(30)

Quantas pessoas moram na casa?            ...           
                                             count      mean  ...   75%  max
Escolaridade                                                  ...           
3° ano EM                                     31.0  3.870968  ...  4.00  6.0
EM completo                                   44.0  4.272727  ...  5.00  7.0
Superior Incompleto                            1.0  5.000000  ...  5.00  5.0
Supletivo                                      2.0  5.500000  ...  5.75  6.0

[4 rows x 8 columns]

In [206]:
NY=(DS.filter(items=['1ºParentesco', '1ºIdade']).groupby(['1ºIdade'] ).describe())

In [207]:
NY.head(30)

1ºParentesco                      
               count unique       top freq
1ºIdade                                   
28.0               1      1   próprio    1
30.0               1      1    Esposo    1
32.0               1      1  Padrasto    1
38.0               2      2       Pai    1
39.0               5      3       Pai    3
40.0               2      2       Pai    1
41.0               1      1       Pai    1
42.0               1      1    Esposo    1
43.0               2      2  padrasto    1
44.0               1      1       mãe    1
46.0               3      2       Pai    2
48.0               1      1       Mãe    1
49.0               3      2       Mãe    2
51.0               2      2       Pai    1
52.0               2      1       Pai    2
53.0               3      2       Pai    2
56.0               2      1        Vó    2
61.0               2      1  padrasto    2

In [210]:
NY2=(DS.filter(items=['2ºParentesco', '2ºIdade']).groupby(['2ºIdade'] ).describe())

In [211]:
NY2.head(30)

2ºParentesco                   
               count unique    top freq
2ºIdade                                
28.0               1      1  Irmão    1
39.0               1      1    Mãe    1
42.0               1      1    Pai    1
51.0               1      1    Pai    1
56.0               2      2   pai     1

In [230]:
NJ=(DS.filter(items=['1ºParentesco', '1ºProfissão/Ocupação','Você tem apoio dos seus familiares? ']).groupby(['1ºProfissão/Ocupação'] ).describe())

In [231]:
NJ.head(30)

1ºParentesco                   ... Você tem apoio dos seus familiares?           
                                    count unique       top  ...                               unique  top freq
1ºProfissão/Ocupação                                        ...                                               
Administrador                           0      0       NaN  ...                                    1  Sim    1
Ajudante Geral                          0      0       NaN  ...                                    1  Sim    2
Ajudante de Produção                    0      0       NaN  ...                                    0  NaN  NaN
Ajudante de Serviços Gerais             1      1       Pai  ...                                    1  Não    1
Algo                                    0      0       NaN  ...                                    0  NaN  NaN
Autonomo                                1      1    Esposo  ...                                    1  Sim    1
Auxiliar de Escritório                  0      0       NaN  ...                                    1  Sim    1
Auxiliar de Manutenção                  0      0       NaN  ...                                    1  Sim    1
Auxiliar de Produção                    0      0       NaN  ...                                    0  NaN  NaN
Auxiliar de Serviços Gerais             2      1        Vó  ...                                    1  Sim    2
Cabeleireira                            1      1       Mãe  ...                                    1  Sim    1
Cabelereira                             1      1       Mãe  ...                                    1  Sim    1
Cervente de Escola                      1      1       Pai  ...                                    1  Sim    1
Costureira                              0      0       NaN  ...                                    1  Sim    1
Cozinheira                              0      0       NaN  ...                                    1  Sim    1
Cuidadora                               1      1       Mãe  ...                                    1  Sim    1
Diagramador                             0      0       NaN  ...                                    1  Sim    1
Diarista                                2      2       Pai  ...                                    1  Sim    2
Doméstica                               1      1       Mãe  ...                                    1  Sim    1
Empregada doméstica                     1      1       mãe  ...                                    1  Não    1
Entregador                              1      1  Padrasto  ...                                    1  Não    1
Estagiária                              0      0       NaN  ...                                    1  Sim    1
Feirante                                0      0       NaN  ...                                    1  Sim    1
Funcionário de fábrica                  2      2       Pai  ...                                    1  Sim    2
Marceneiro                              1      1       Pai  ...                                    1  Não    1
Monitora                                0      0       NaN  ...                                    0  NaN  NaN
Motorista                               2      1       Pai  ...                                    1  Sim    2
Operador de Produção                    0      0       NaN  ...                                    1  Sim    1
Operadora de Loja                       0      0       NaN  ...                                    1  Sim    1
Padeiro                                 1      1       Pai  ...                                    0  NaN  NaN

[30 rows x 8 columns]

In [233]:
fig = px.histogram(DadosA, x="Quantas pessoas moram na casa?")
fig.show()


In [162]:
#DadosA['Instituição de preferência'].value_counts()
#DadosA['Você tem apoio dos seus familiares? '].value_counts()
#DadosA['Escolaridade'].value_counts()
#DadosA['Quantas pessoas moram na casa?'].value_counts()



EM completo            44
3° ano EM              31
Supletivo               2
Superior Incompleto     1
Name: Escolaridade, dtype: int64